In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    """
    S: Current stock price
    K: Strike price
    T: Time to maturity in years
    r: Risk-free interest rate (annualized)
    sigma: Volatility of the underlying asset (annualized)
    option_type: 'call' or 'put'
    """
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 - r * K * np.exp(-r * T) * norm.cdf(d2))
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 + r * K * np.exp(-r * T) * norm.cdf(-d2))
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("option_type must be either 'call' or 'put'")
    
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)

    return {
        'price': price,
        'delta': delta,
        'gamma': gamma,
        'vega': vega / 100,  # per 1% change
        'theta': theta / 365,  # per day
        'rho': rho / 100  # per 1% change
    }


In [3]:
# Example Parameters
S = 100      # Current stock price
K = 100      # Strike price
T = 1        # 1 year to expiry
r = 0.05     # 5% risk-free rate
sigma = 0.2  # 20% annual volatility

# Call Option
result_call = black_scholes(S, K, T, r, sigma, 'call')
print("Call Option:", result_call)

# Put Option
result_put = black_scholes(S, K, T, r, sigma, 'put')
print("Put Option:", result_put)

Call Option: {'price': 10.450583572185565, 'delta': 0.6368306511756191, 'gamma': 0.018762017345846895, 'vega': 0.3752403469169379, 'theta': -0.01757267820941972, 'rho': 0.5323248154537634}
Put Option: {'price': 5.573526022256971, 'delta': -0.3631693488243809, 'gamma': 0.018762017345846895, 'vega': 0.3752403469169379, 'theta': -0.004542138147766099, 'rho': -0.4189046090469506}
